In [1]:
%gui qt

from OCC.Display.SimpleGui import init_display
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_FACE, TopAbs_VERTEX
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface, BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Plane, GeomAbs_Cylinder

from utils_geometry import *
from utils_dstv import *
from utils_visualization import *
from utils_classification import *
from utils_reports import *

import numpy as np
import json

DSTV_FACE_MAP = {'I':['O','U','V'], 'U':['H','U','O'], 'L':['H','U']}

# Start display without blocking
display, start_display, add_menu, add_function_to_menu = init_display()

display.EraseAll()
display.View_Iso()
display.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
from OCC.Extend.DataExchange import read_step_file
# shape = read_step_file("../data/0444-1 ANGLED.step")
# shape = read_step_file("../data/ncTest.step")
# shape = read_step_file("../data/TestEA.step")
# shape = read_step_file("../data/TestEAMirror.step")
# shape = read_step_file("../data/TestUEA.step")
# shape = read_step_file("../data/TestUEAMirror.step")
shape = read_step_file("../data/TestPFC.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [3]:
# Utilities to compute DSTV origins and axes from an actual section face

from OCC.Core.gp import gp_Vec, gp_Pnt, gp_Dir, gp_Pln
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
import numpy as np

def make_section_face(solid, origin, normal):
    plane = gp_Pln(origin, gp_Dir(normal))
    section = BRepAlgoAPI_Section(solid, plane)
    section.ComputePCurveOn1(True)
    section.Approximation(True)
    section.Build()

    wire_builder = BRepBuilderAPI_MakeWire()
    exp = TopExp_Explorer(section.Shape(), TopAbs_EDGE)
    while exp.More():
        wire_builder.Add(topods.Edge(exp.Current()))
        exp.Next()

    return BRepBuilderAPI_MakeFace(wire_builder.Wire()).Face()

def get_section_face_corners(face):
    exp = TopExp_Explorer(face, TopAbs_VERTEX)
    corners = []
    while exp.More():
        v = topods.Vertex(exp.Current())
        pnt = BRep_Tool.Pnt(v)
        corners.append(pnt)
        exp.Next()
    return corners

def order_corners_by_local_axes(corners, xaxis, yaxis):
    # Project points to local X/Y plane and sort
    def project(pnt):
        return np.array([pnt.X(), pnt.Y(), pnt.Z()])

    pts = [project(p) for p in corners]
    origin = np.mean(pts, axis=0)
    x_dir = np.array([xaxis.X(), xaxis.Y(), xaxis.Z()])
    y_dir = np.array([yaxis.X(), yaxis.Y(), yaxis.Z()])

    local_coords = []
    for pt in pts:
        rel = pt - origin
        lx = np.dot(rel, x_dir)
        ly = np.dot(rel, y_dir)
        local_coords.append((lx, ly, pt))

    # Sort by Y then X
    local_coords.sort(key=lambda v: (-v[1], v[0]))  # Top-left, top-right, bottom-left, bottom-right
    return [gp_Pnt(*v[2]) for v in local_coords]

def get_face_area(face):
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    return props.Mass()

def compute_dstv_origins_and_axes_from_section(solid, obb_center, he_Z, xaxis, yaxis, zaxis, profile_type):
    
    # Step 1: Try both ends and choose larger
    end1 = gp_Pnt(obb_center.X() + he_Z * zaxis.X(),
                  obb_center.Y() + he_Z * zaxis.Y(),
                  obb_center.Z() + he_Z * zaxis.Z())
    end2 = gp_Pnt(obb_center.X() - he_Z * zaxis.X(),
                  obb_center.Y() - he_Z * zaxis.Y(),
                  obb_center.Z() - he_Z * zaxis.Z())

    face1 = make_section_face(solid, end1, zaxis)
    face2 = make_section_face(solid, end2, zaxis)

    a1 = get_face_area(face1)
    a2 = get_face_area(face2)
    print(f"Face area → end1: {a1:.1f}, end2: {a2:.1f}")

    if a2 > a1:
        print("Swapping Z axis to align with larger end face")
        zaxis = zaxis.Reversed()
        face = face2
        face_origin = end2
    else:
        face = face1
        face_origin = end1

    # Step 2: Get ordered corners and assign DSTV origins
    corners = get_section_face_corners(face)
    if len(corners) < 3:
        print("⚠️ Not enough corners found on section face")
        return {}, {}

    ordered = order_corners_by_local_axes(corners, xaxis, yaxis)
    origin_v = ordered[2]  # bottom-left
    origin_o = ordered[1]  # top-right
    origin_u = ordered[3]  # bottom-right

    origins = {'V': origin_v, 'O': origin_o, 'U': origin_u}

    # Step 3: Flip X axis if angle is unequal and longer leg is not in U
    if profile_type == "L":
        vec_vo = np.array([origin_o.X() - origin_v.X(), origin_o.Y() - origin_v.Y(), origin_o.Z() - origin_v.Z()])
        vec_vu = np.array([origin_u.X() - origin_v.X(), origin_u.Y() - origin_v.Y(), origin_u.Z() - origin_v.Z()])

        len_o = np.linalg.norm(vec_vo)
        len_u = np.linalg.norm(vec_vu)

        if len_o > len_u:
            print("Angle leg in O is longer → swapping origins for U and O")
            origins['U'], origins['O'] = origins['O'], origins['U']
            xaxis = gp_Dir(-xaxis.X(), -xaxis.Y(), -xaxis.Z())

    face_axes = {
        'V': (gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z()), gp_Vec(yaxis)),
        'O': (gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z()), gp_Vec(xaxis)),
        'U': (gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z()), gp_Vec(-xaxis.X(), -xaxis.Y(), -xaxis.Z()))
    }

    return origins, face_axes, face


In [4]:

def fingerprint_shape(solid, obb_dims):
    """
    Analyze face normals and areas to guess profile type: Beam, Channel, Angle, Unknown
    """
    face_areas = []
    face_normals = []

    exp = TopExp_Explorer(solid, TopAbs_FACE)
    while exp.More():
        face = topods.Face(exp.Current())
        surf = BRepAdaptor_Surface(face)
        if surf.GetType() != GeomAbs_Plane:
            exp.Next()
            continue

        # Normal
        n = surf.Plane().Axis().Direction()
        n = np.array([n.X(), n.Y(), n.Z()], dtype=float)
        n /= np.linalg.norm(n)

        # Area
        gp = GProp_GProps()
        brepgprop.SurfaceProperties(face, gp)
        area = gp.Mass()

        face_normals.append(n)
        face_areas.append(area)
        exp.Next()

    # Sort OBB dimensions
    dims = sorted(obb_dims)  # [flange_thickness, web_depth, length]
    thickness, height, length = dims
    slenderness = length / height if height else 0
    aspect_ratio = height / thickness if thickness else 0

    # Area stats
    largest = max(face_areas) if face_areas else 0
    large_faces = [a for a in face_areas if a > 0.8 * largest]

    print("\n🔎 Shape fingerprint debug:")
    print(f"  OBB dimensions (sorted): {np.round(dims,1)}")
    print(f"  Slenderness ratio (L/H): {slenderness:.2f}")
    print(f"  Aspect ratio (H/t):      {aspect_ratio:.2f}")
    print(f"  Total planar faces:      {len(face_areas)}")
    print(f"  Large planar faces:      {len(large_faces)}")
    print(f"  Max area:                {max(face_areas):.1f}" if face_areas else "  Max area: —")
    print(f"  Large face areas:        {[round(a,1) for a in large_faces]}")
    
    # Fingerprint logic
    
    # --- Check for Beam ---
    if len(face_areas) >= 10 and slenderness > 4 and max(face_areas) > 100000:
        return "I"
    
    # --- Check for Angles ---
    elif len(large_faces) == 2 and slenderness > 6:
        a1, a2 = large_faces
        area_ratio = max(a1, a2) / min(a1, a2)
        if area_ratio < 1.1:
            return "L"
        else:
            return "L"

    # --- Check for Channel ---
    elif len(face_areas) >= 6 and slenderness > 6 and max(face_areas) > 50000:
        return "U"
        
    # --- Fallback ---
    return "Unknown"


In [5]:
# from OCC.Core.gp import gp_Pnt, gp_Vec
# from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

# def draw_global_origin_axes(display, length=200.0):
#     origin = gp_Pnt(0, 0, 0)

#     # X (Red)
#     x_end = gp_Pnt(length, 0, 0)
#     x_edge = BRepBuilderAPI_MakeEdge(origin, x_end).Edge()
#     display.DisplayShape(x_edge, color="RED", update=False)

#     # Y (Green)
#     y_end = gp_Pnt(0, length, 0)
#     y_edge = BRepBuilderAPI_MakeEdge(origin, y_end).Edge()
#     display.DisplayShape(y_edge, color="GREEN", update=False)

#     # Z (Blue)
#     z_end = gp_Pnt(0, 0, length)
#     z_edge = BRepBuilderAPI_MakeEdge(origin, z_end).Edge()
#     display.DisplayShape(z_edge, color="BLUE", update=True)

# # Call it
# # draw_global_origin_axes(display)


In [6]:

# Display solid
display.DisplayShape(solid, update=True)

# Compute OBB
obb_data = compute_obb_and_local_axes(solid)

local_cs    = obb_data['local_cs']
center       = obb_data['center']
xaxis        = obb_data['xaxis']
yaxis        = obb_data['yaxis']
zaxis        = obb_data['zaxis']
xdir        = obb_data['xaxis_dir']
ydir        = obb_data['yaxis_dir']
zdir        = obb_data['zaxis_dir']
he_X         = obb_data['he_X']
he_Y         = obb_data['he_Y']
he_Z         = obb_data['he_Z']
half_extents = [he_X, he_Y, he_Z]
obb_dims     = [2*he_X, 2*he_Y, 2*he_Z]

# Draw bounding box
draw_obb(display,
         center=obb_data['center'],
         xaxis=obb_data['xaxis'],
         yaxis=obb_data['yaxis'],
         zaxis=obb_data['zaxis'],
         he_X=obb_data['he_X'],
         he_Y=obb_data['he_Y'],
         he_Z=obb_data['he_Z'],
         color="YELLOW")

# Identify profile type
profile_type = fingerprint_shape(shape, obb_dims)
print(f"\nProfile Type is : {profile_type}")

# Identify element - use classifier against library
section_face, section_origin = make_section_face_at_start(solid, center, zaxis, he_Z)
cs = prepare_classification_input(obb_data, profile_type, section_face)
matched_profile = classify_profile(cs, "../data/Shape_classifier_info.json")

print(matched_profile)
print(f"\n🔍 Best match: {matched_profile['Designation']} in category {matched_profile['Category']}")

if "Swapped_dimensions" in matched_profile:
    print("⚠️  Matched after swapping width/height — OBB may be misaligned.")
    xdir, ydir = ydir, xdir
    xaxis, yaxis = yaxis, xaxis
    he_X, he_Y = he_Y, he_X
    local_cs = gp_Ax3(center, zdir, xdir)
display.FitAll()

ordered_corners = extract_ordered_section_corners(
    face=section_face,
    xaxis=xaxis,
    yaxis=yaxis,
    origin=section_origin
)

if profile_type == "L":
    z_flipped = False
    if profile_type == "L":
        z_flipped = should_flip_zaxis_for_angle(ordered_corners, xaxis, yaxis, section_origin)
        if z_flipped:
            zaxis_dir = zaxis_dir.Reversed()
            zaxis = zaxis.Reversed()

# report_summary(
#     obb_data=obb_data,
#     matched_profile=matched_profile,
#     profile_type=profile_type,
#     section_info={
#         "z_cut": "start",
#         "corners": ordered_corners,
#         "z_flipped": z_flipped,
#         "leg_orientation_ok": True
#     },
#     dstv_info={
#         "origins": origins,
#         "face_axes": face_axes
#     }
# )

center           : <class 'gp_Pnt'> type: <class 'OCC.Core.gp.gp_Pnt'>
raw_axes[wi]     : [<class 'gp_XYZ'>, <class 'gp_XYZ'>, <class 'gp_XYZ'>] type: <class 'list'>
xaxis_vec        : <class 'gp_Vec'> type: <class 'OCC.Core.gp.gp_Vec'>
xaxis_dir        : <class 'gp_Dir'> type: <class 'OCC.Core.gp.gp_Dir'>
zaxis_vec        : <class 'gp_Vec'> type: <class 'OCC.Core.gp.gp_Vec'>
zaxis_dir        : <class 'gp_Dir'> type: <class 'OCC.Core.gp.gp_Dir'>
extents sorted   : 2 1 0
half-extents     : (25.000000000000018, 50.0, 500.0)

🔎 Shape fingerprint debug:
  OBB dimensions (sorted): [  50.  100. 1000.]
  Slenderness ratio (L/H): 10.00
  Aspect ratio (H/t):      2.00
  Total planar faces:      10
  Large planar faces:      1
  Max area:                99921.5
  Large face areas:        [99921.5]

Profile Type is : U
{'mass': 10.20315915547705, 'height': 100.0, 'width': 50.0, 'csa': 1300.0, 'web_thickness': 5.0, 'flange_thickness': 8.5, 'root_radius': 9.0, 'toe_radius': 0.0, 'code_profile': 'U'

In [7]:
# Add shitty legend to visualization
draw_legend(display, origin=(0, -50, -200))  # Offset in -Y direction from part

Many colors for color name BLUE, using first.


In [8]:
# Create face axis from matched section size, then display on view
face_axes = get_aligned_axes_from_profile(obb_data, matched_profile)
face_data = visualize_dstv_faces(display, obb_data, face_axes, local_cs, offset=10.0)

draw_local_axes(
    display,
    center=obb_data["center"],
    xaxis=face_axes["x"],        # show U face direction as local X
    yaxis=face_axes["y"],        # show V face direction as local Y
    zaxis=face_axes["z"],    # show START face direction as local Z
    length=25
)


Many colors for color name BLUE, using first.
Many colors for color name CYAN, using first.
Many colors for color name BLUE, using first.


In [9]:
# df_holes_by_face = get_projected_holes_by_face_drill_direction(
#     solid = solid, 
#     section_type = profile_type, 
#     face_data = face_data,
#     obb_data = obb_data,
#     display = display,
#     threshold=0.98)

# print(df_holes_by_face)

# for index, row in df_holes_by_face.iterrows():
#     center = gp_Pnt(row["X (mm)"], row["Y (mm)"], 0)  # Assumes projected flat view
#     display.DisplayShape(center, color='YELLOW', update=False)
# display.FitAll()

In [10]:
# df_cylinders = extract_cylinders_local(solid, obb_data)
# print(df_cylinders)
# visualize_cylinder_debug(display, df_cylinders, obb_data)

In [11]:
# df_projected_holes = project_cylinders_onto_faces(
#     solid=solid,
#     obb_data=obb_data,
#     face_data=face_data,
#     section_type=profile_type
# )

# print(df_projected_holes)

# visualize_projected_holes(display, df_projected_holes, obb_data)

In [12]:
# df_assigned = assign_holes_to_faces_from_solid(
#     solid=solid,
#     obb_data=obb_data,
#     face_data=face_data,
#     section_type=profile_type,
#     tol=0.8
# )
# print(df_assigned)
# visualize_assigned_holes(display, df_assigned, obb_data)

In [13]:
# df_holes_sectioned = section_holes_on_dstv_faces(
#     solid=solid,
#     obb_data=obb_data,
#     face_data=face_data,
#     section_type=profile_type
# )
# print(df_holes_sectioned)
# visualize_assigned_holes(display, df_holes_sectioned, obb_data)

In [14]:
# from OCC.Core.gp import gp_Dir, gp_Pnt

# # Suppose you found:
# center = center
# normal = zdir  # e.g. from Geom_Plane.Axis().Direction()
# radius =10

# ais_hole = show_hole_marker(
#     display,
#     center,
#     normal,
#     radius,
#     axis_scale=2.0,
#     color=(0.0, 0.8, 0.2),
#     width=2.5,
#     transparency=0.3
# )

In [15]:
# import pandas as pd
# from OCC.Core.TopExp import TopExp_Explorer
# from OCC.Core.TopAbs import TopAbs_FACE
# from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
# from OCC.Core.GeomAbs import GeomAbs_Cylinder
# from OCC.Core.gp import gp_Trsf, gp_Ax3, gp_Vec, gp_Dir
# from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

# def identify_cylindrical_holes_local(solid, display, local_cs, show_hole_marker, axis_scale=2.0):
#     """
#     Identify cylindrical holes in `solid`, transform them into `local_cs`,
#     display them with `show_hole_marker`, and return a DataFrame with:
#       hole#, face_id, dia, x, y, z (all x,y,z in local CS)
#     """
#     records = []
#     explorer = TopExp_Explorer(solid, TopAbs_FACE)
#     hole_no  = 0
#     face_id  = 0

#     # Build a transform from global CS → local_cs
#     trsf = gp_Trsf()
#     trsf.SetTransformation(gp_Ax3(), local_cs)

#     while explorer.More():
#         face_id += 1
#         face = explorer.Current()

#         # Adapt the face to see if it’s a cylinder
#         adaptor = BRepAdaptor_Surface(face)
#         if adaptor.GetType() == GeomAbs_Cylinder:
#             hole_no += 1

#             # Extract cylinder parameters
#             cyl     = adaptor.Cylinder()
#             radius  = cyl.Radius()
#             dia     = 2.0 * radius
#             axis    = cyl.Axis()               # gp_Ax1
#             origin  = axis.Location()          # gp_Pnt
#             direction = axis.Direction()       # gp_Dir

#             # Transform origin & direction into local CS
#             origin_local    = origin.Transformed(trsf)
#             v_local         = gp_Vec(direction)
#             v_local.Transform(trsf)
#             direction_local = gp_Dir(v_local)

#             # Show the marker in local CS
#             show_hole_marker(
#                 display,
#                 origin_local,
#                 direction_local,
#                 radius,
#                 axis_scale=axis_scale
#             )

#             # Record the local coordinates
#             x, y, z = origin_local.Coord()
#             records.append({
#                 'hole#':   hole_no,
#                 'face_id': face_id,
#                 'dia':     dia,
#                 'x':       x,
#                 'y':       y,
#                 'z':       z
#             })

#         explorer.Next()

#     # Build the DataFrame
#     df = pd.DataFrame(records, columns=['hole#','face_id','dia','x','y','z'])
#     return df

# # 1) Run the hole-finder
# df_holes = identify_cylindrical_holes_local(
#     solid,
#     display,
#     local_cs,
#     show_hole_marker,   # your helper from before
#     axis_scale=2.5
# )

# # 3) Inspect the results
# print(df_holes)


In [16]:

# 1) Run the hole-finder
df_holes = identify_cylindrical_holes_local(
    solid,
    display,
    local_cs,
    show_hole_marker,   # your helper from before
    axis_scale=2.5
)

# 2) Inspect the results
print(df_holes)


   hole#  face_id   dia             x             y      z            nx  \
0      1        1  10.0 -1.065814e-14 -4.150000e+01 -400.0  3.552714e-16   
1      2        2  10.0 -1.065814e-14 -5.000000e+01 -450.0 -3.552714e-16   
2      3        3  10.0 -3.552714e-15  4.150000e+01 -450.0 -3.552714e-16   
3      4        4  10.0  2.000000e+01 -1.188571e-14 -400.0 -1.000000e+00   
4      5        5  18.0  1.100000e+01  3.250000e+01 -500.0  0.000000e+00   
5      6       13  18.0  1.100000e+01 -3.250000e+01 -500.0  0.000000e+00   

             ny   nz  
0 -1.000000e+00 -0.0  
1 -1.000000e+00  0.0  
2 -1.000000e+00  0.0  
3  4.440892e-17  0.0  
4  0.000000e+00  1.0  
5  0.000000e+00  1.0  


In [17]:
import numpy as np
import pandas as pd

def project_holes_to_planes(
    df_holes: pd.DataFrame,
    planes: list[dict],
    profile_type: str,
    face_map: dict[str, list[str]],
    max_dist: float = 10.0,
    angle_tol_deg: float = 15.0
) -> pd.DataFrame:
    """
    Projects each hole in df_holes onto the DSTV planes allowed by profile_type.

    Parameters
    ----------
    df_holes : DataFrame with columns ['x','y','z','nx','ny','nz']
    planes   : list of dicts with keys:
                   'id'     : plane identifier (e.g. 'O','U','V','H','start')
                   'origin' : tuple (x,y,z)
                   'normal' : tuple (nx,ny,nz)
    profile_type : one of the keys in face_map (e.g. 'I','U','L')
    face_map  : dict mapping profile_type → list of plane IDs to use
    max_dist  : maximum allowed distance from hole center to plane
    angle_tol_deg : maximum allowed angle (in degrees) between hole axis and plane normal

    Returns
    -------
    DataFrame with columns:
      ['hole_idx','plane_id','proj_x','proj_y','proj_z','dist','angle_deg']
    """
    cos_tol = np.cos(np.deg2rad(angle_tol_deg))
    allowed = set(face_map.get(profile_type, []))
    if not allowed:
        raise ValueError(f"No DSTV planes mapped for profile_type={profile_type!r}")

    recs = []
    for idx, hole in df_holes.iterrows():
        p = np.array([hole.x, hole.y, hole.z], dtype=float)
        nh = np.array([hole.nx, hole.ny, hole.nz], dtype=float)
        nh /= np.linalg.norm(nh)

        for pl in planes:
            pid = pl['id']
            if pid not in allowed:
                continue

            origin = np.array(pl['origin'], dtype=float)
            normal = np.array(pl['normal'], dtype=float)
            normal /= np.linalg.norm(normal)

            # 1) signed distance
            d = np.dot(p - origin, normal)
            if abs(d) > max_dist:
                continue

            # 2) angle check
            dotn = np.dot(nh, normal)
            if abs(dotn) < cos_tol:
                continue

            # 3) projection
            proj = p - d * normal

            recs.append({
                'hole_idx':  idx,
                'plane_id':  pid,
                'proj_x':    proj[0],
                'proj_y':    proj[1],
                'proj_z':    proj[2],
                'dist':      d,
                'angle_deg': np.degrees(np.arccos(abs(dotn)))
            })

    return pd.DataFrame(recs)


In [18]:
# # 1) Compute your local DSTV axes from the OBB + profile
# # face_axes = get_aligned_axes_from_profile(obb_data, matched_profile)

# # 2) Draw the START, U, O, V, H faces AND collect their origin+normal in face_data
# # face_data = visualize_dstv_faces(display, obb_data, face_axes, offset=10.0)

# # 3) Turn that face_data into the simple list-of-dicts our projector wants
# #    (and filter to only the faces relevant for your profile_type)
# planes = [
#     {
#       'id':    pid,
#       'origin': info['origin'],
#       'normal': info['normal']
#     }
#     for pid, info in face_data.items()
#     if pid in DSTV_FACE_MAP[profile_type]
# ]

# #---------- Debug

# dbg = []
# for idx, hole in df_holes.iterrows():
#     p  = np.array([hole.x, hole.y, hole.z], dtype=float)
#     nh = np.array([hole.nx, hole.ny, hole.nz], dtype=float)
#     nh /= np.linalg.norm(nh)
#     for pl in planes:
#         origin = np.array(pl['origin'], dtype=float)
#         normal = np.array(pl['normal'], dtype=float)
#         normal /= np.linalg.norm(normal)
#         d    = np.dot(p - origin, normal)
#         dotn = np.dot(nh, normal)
#         dbg.append({
#             'hole_idx': idx,
#             'plane_id': pl['id'],
#             'dist':     d,
#             'abs_dist': abs(d),
#             'dotn':     dotn,
#             'angle':    np.degrees(np.arccos(abs(dotn)))
#         })

# df_debug = pd.DataFrame(dbg)
# print(df_debug)

# #---------- Debug

# # 4) Detect all the holes in the solid
# # df_holes = identify_cylindrical_holes_local(solid)

# # 5) Project those holes onto the allowed DSTV planes
# df_proj = project_holes_to_planes(
#     df_holes,
#     planes,
#     profile_type,
#     DSTV_FACE_MAP,
#     max_dist=15.0,
#     angle_tol_deg=15.0
# )

# print(df_proj)

# # 6) Finally, draw each projected centre in green
# from OCC.Core.gp       import gp_Pnt
# from OCC.Core.Geom     import Geom_CartesianPoint
# from OCC.Core.AIS      import AIS_Point
# from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB

# green = Quantity_Color(0,1,0, Quantity_TOC_RGB)
# for _, row in df_proj.iterrows():
#     pnt    = gp_Pnt(row['proj_x'], row['proj_y'], row['proj_z'])
#     geom   = Geom_CartesianPoint(pnt)
#     marker = AIS_Point(geom)
#     display.Context.SetColor(marker, green, True)
#     display.Context.Display(marker, True)

# display.FitAll()


In [19]:
# # 1) Compute local DSTV axes and draw faces + collect local data
# face_axes = get_aligned_axes_from_profile(obb_data, matched_profile)
# face_data = visualize_dstv_faces(display, obb_data, face_axes, local_cs, offset=10.0)

# 2) Build planes list in local coords
planes = [
    {'id': pid, 'origin': info['origin'], 'normal': info['normal']}
    for pid, info in face_data.items()
    if pid in DSTV_FACE_MAP[profile_type]
]

# 3) Detect holes (they’re already in local)
# df_holes = identify_cylindrical_holes_local(solid, display, local_cs, show_hole_marker)

# 4) Project in local and display green points
df_proj = project_holes_to_planes(
    df_holes,
    planes,
    profile_type,
    DSTV_FACE_MAP,
    max_dist=10.0,
    angle_tol_deg=15.0
)

# 5) Visualize
from OCC.Core.gp       import gp_Pnt
from OCC.Core.Geom     import Geom_CartesianPoint
from OCC.Core.AIS      import AIS_Point
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB

green = Quantity_Color(0,1,0, Quantity_TOC_RGB)
for _, row in df_proj.iterrows():
    pnt  = gp_Pnt(row['proj_x'], row['proj_y'], row['proj_z'])
    geom = Geom_CartesianPoint(pnt)
    marker = AIS_Point(geom)
    display.Context.SetColor(marker, green, True)
    display.Context.Display(marker, True)

display.FitAll()
